# 1 Matrix operations

## 1.1 Create a 4*4 identity matrix

In [1]:
#This project is designed to get familiar with python list and linear algebra
#You cannot use import any library yourself, especially numpy

A = [[1,2,3], 
     [2,3,3], 
     [1,2,5]]

B = [[1,2,3,5], 
     [2,3,3,5], 
     [1,2,5,1]]

#TODO create a 4*4 identity matrix 
I = [[1 if col == row else 0 for col in range(4)] for row in range(4)]

## 1.2 get the width and height of a matrix. 

In [2]:
#TODO Get the height and weight of a matrix.
def shape(M):
    return len(M), len(M[0])

In [3]:
# run following code to test your shape function
%run -i -e test.py LinearRegressionTestCase.test_shape

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## 1.3 round all elements in M to certain decimal points

In [4]:
# TODO in-place operation, no return value
# TODO round all elements in M to decPts
def matxRound(M, decPts=4):
    for row in range(len(M)):
        for col in range(len(M[0])):
            M[row][col] = round(M[row][col], decPts)

In [5]:
# run following code to test your matxRound function
%run -i -e test.py LinearRegressionTestCase.test_matxRound

.
----------------------------------------------------------------------
Ran 1 test in 0.079s

OK


## 1.4 compute transpose of M

In [6]:
#TODO compute transpose of M
def transpose(M):
    return [[M[row][col] for row in range(len(M))] for col in range(len(M[0]))]

In [7]:
# run following code to test your transpose function
%run -i -e test.py LinearRegressionTestCase.test_transpose

.
----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


## 1.5 compute AB. return None if the dimensions don't match

In [8]:
#TODO compute matrix multiplication AB, return None if the dimensions don't match
def matxMultiply(A, B):
    if len(A[0]) != len(B):
        return None
    return [[sum(a * b for a, b in zip(a, b)) for b in zip(*B)] for a in A]

In [9]:
# run following code to test your matxMultiply function
%run -i -e test.py LinearRegressionTestCase.test_matxMultiply

.
----------------------------------------------------------------------
Ran 1 test in 0.061s

OK


---

# 2 Gaussian Jordan Elimination

## 2.1 Compute augmented Matrix 

$ A = \begin{bmatrix}
    a_{11}    & a_{12} & ... & a_{1n}\\
    a_{21}    & a_{22} & ... & a_{2n}\\
    a_{31}    & a_{22} & ... & a_{3n}\\
    ...    & ... & ... & ...\\
    a_{n1}    & a_{n2} & ... & a_{nn}\\
\end{bmatrix} , b = \begin{bmatrix}
    b_{1}  \\
    b_{2}  \\
    b_{3}  \\
    ...    \\
    b_{n}  \\
\end{bmatrix}$

Return $ Ab = \begin{bmatrix}
    a_{11}    & a_{12} & ... & a_{1n} & b_{1}\\
    a_{21}    & a_{22} & ... & a_{2n} & b_{2}\\
    a_{31}    & a_{22} & ... & a_{3n} & b_{3}\\
    ...    & ... & ... & ...& ...\\
    a_{n1}    & a_{n2} & ... & a_{nn} & b_{n} \end{bmatrix}$

In [10]:
#TODO construct the augment matrix of matrix A and column vector b, assuming A and b have same number of rows
def augmentMatrix(A, b):
    return [a_row + b_row for a_row, b_row in zip(A, b)]

In [11]:
# run following code to test your augmentMatrix function
%run -i -e test.py LinearRegressionTestCase.test_augmentMatrix

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


## 2.2 Basic row operations
- exchange two rows
- scale a row
- add a scaled row to another

In [12]:
# TODO r1 <---> r2
# TODO in-place operation, no return value
def swapRows(M, r1, r2):
    for i in range(len(M[r1])):
        M[r1][i], M[r2][i] = M[r2][i], M[r1][i]

In [13]:
# run following code to test your swapRows function
%run -i -e test.py LinearRegressionTestCase.test_swapRows

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [14]:
# TODO r1 <--- r1 * scale
# TODO in-place operation, no return value
def scaleRow(M, r, scale):
    if scale == 0:
        raise ValueError
    for i in range(len(M[r])):
        M[r][i] *= scale

In [15]:
# run following code to test your scaleRow function
%run -i -e test.py LinearRegressionTestCase.test_scaleRow

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [16]:
# TODO r1 <--- r1 + r2*scale
# TODO in-place operation, no return value
def addScaledRow(M, r1, r2, scale):
    for i in range(len(M[r1])):
        M[r1][i] += M[r2][i] * scale

In [17]:
# run following code to test your addScaledRow function
%run -i -e test.py LinearRegressionTestCase.test_addScaledRow

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## 2.3  Gauss-jordan method to solve Ax = b

### Hint：

Step 1: Check if A and b have same number of rows
Step 2: Construct augmented matrix Ab

Step 3: Column by column, transform Ab to reduced row echelon form [wiki link](https://en.wikipedia.org/wiki/Row_echelon_form#Reduced_row_echelon_form)
    
    for every column of Ab (except the last one)
        column c is the current column
        Find in column c, at diagonal and under diagonal (row c ~ N) the maximum absolute value
        If the maximum absolute value is 0
            then A is singular, return None （Prove this proposition in Question 2.4）
        else
            Apply row operation 1, swap the row of maximum with the row of diagonal element (row c)
            Apply row operation 2, scale the diagonal element of column c to 1
            Apply row operation 3 mutiple time, eliminate every other element in column c
            
Step 4: return the last column of Ab

### Remark：
We don't use the standard algorithm first transfering Ab to row echelon form and then to reduced row echelon form.  Instead, we arrives directly at reduced row echelon form. If you are familiar with the stardard way, try prove to yourself that they are equivalent. 

In [18]:
#TODO implement gaussian jordan method to solve Ax = b

""" Gauss-jordan method to solve x such that Ax = b.
        A: square matrix, list of lists
        b: column vector, list of lists
        decPts: degree of rounding, default value 4
        epsilon: threshold for zero, default value 1.0e-16
        
    return x such that Ax = b, list of lists 
    return None if A and b have same height
    return None if A is (almost) singular
"""

def find_maximum(M, col, start_idx):
    max_val, max_idx = 0, 0
    for row in range(len(M)):
        if row >= start_idx:
            if abs(M[row][col]) > max_val:
                max_val = abs(M[row][col])
                max_idx = row
    return max_val, max_idx
            

def gj_Solve(A, b, decPts=4, epsilon = 1.0e-16):
    if len(A) != len(b):
        return None
    M = augmentMatrix(A, b)

    for col in range(len(M[0])-1):
        max_val, max_idx = find_maximum(M, col, col)
        if max_val == 0:
            return None
        else:
            swapRows(M, r1=max_idx, r2=col)
            scaleRow(M, col, 1.0 / M[col][col])
            for row in range(len(M)):
                if row != col:
                    addScaledRow(M, r1=row, r2=col, scale=-M[row][col])
    return [[M[row][len(M[0])-1]] for row in range(len(M))]

In [19]:
# run following code to test your addScaledRow function
%run -i -e test.py LinearRegressionTestCase.test_gj_Solve

.
----------------------------------------------------------------------
Ran 1 test in 0.047s

OK


## 2.4 Prove the following proposition:

**If square matrix A can be divided into four parts: ** 

$ A = \begin{bmatrix}
    I    & X \\
    Z    & Y \\
\end{bmatrix} $, where I is the identity matrix, Z is all zero and the first column of Y is all zero, 

**then A is singular.**

Hint: There are mutiple ways to prove this problem.  
- consider the rank of Y and A
- consider the determinate of Y and A 
- consider certain column is the linear combination of other columns

TODO Please use latex （refering to the latex in problem may help）

TODO Proof：

1. We know `A` is a square matrix, so its number of rows and number of columns are same
2. Then if `|A| = 0`, `A` will be singular matrix
3. Because `I` is identity matrix, `Z` is all zero and the first column of `Y` is all zero, matrix `A` is an upper triangular matrix
4. For an upper triangular matrix, the determinant will equal to the multiply of elements in diagonal
5. Because the first column of `Y` is all zero, there will be `0` in one of the diagonal elements
6. `|A| = 0`, `A` is singular

---

# 3 Linear Regression: 

## 3.1 Compute the gradient of loss function with respect to parameters 
## (Choose one between two 3.1 questions)

We define loss funtion $E$ as 
$$
E(m, b) = \sum_{i=1}^{n}{(y_i - mx_i - b)^2}
$$
and we define vertex $Y$, matrix $X$ and vertex $h$ :
$$
Y =  \begin{bmatrix}
    y_1 \\
    y_2 \\
    ... \\
    y_n
\end{bmatrix}
,
X =  \begin{bmatrix}
    x_1 & 1 \\
    x_2 & 1\\
    ... & ...\\
    x_n & 1 \\
\end{bmatrix},
h =  \begin{bmatrix}
    m \\
    b \\
\end{bmatrix}
$$


Proves that 
$$
\frac{\partial E}{\partial m} = \sum_{i=1}^{n}{-2x_i(y_i - mx_i - b)}
$$

$$
\frac{\partial E}{\partial b} = \sum_{i=1}^{n}{-2(y_i - mx_i - b)}
$$

$$
\begin{bmatrix}
    \frac{\partial E}{\partial m} \\
    \frac{\partial E}{\partial b} 
\end{bmatrix} = 2X^TXh - 2X^TY
$$

TODO Please use latex （refering to the latex in problem may help）

TODO Proof：
$$
E(m, b) = \sum_{i=1}^{n}{(y_i - mx_i - b)^2} = \sum_{i=1}^{n}{(y_i)^2 + (mx_i)^2 + b^2 - 2mx_iy_i - 2by_i + 2mx_ib}
$$

$$
\star \;\;\; \frac{\partial E}{\partial m} = \sum_{i=1}^{n}{2mx_i^2 - 2x_iy_i + 2x_ib} = \sum_{i=1}^{n}{-2x_i(y_i - mx_i - b)}
$$

$$
\star \;\;\; \frac{\partial E}{\partial b} = \sum_{i=1}^{n}{2b - 2y_i + 2mx_i} = \sum_{i=1}^{n}{-2(y_i - mx_i - b)}
$$

$$
X^TX =  \begin{bmatrix}
    \sum{x_i^2} & \sum{x_i} \\
    \sum{x_i} & n \\
\end{bmatrix}
$$

$$
2X^TXh = \begin{bmatrix}
    2m\sum{x_i^2} + 2b\sum{x_i} \\
    2m\sum{x_i} + 2bn \\
\end{bmatrix}
$$

$$
2X^TY = \begin{bmatrix}
    2\sum{x_iy_i} \\
    2\sum{y_i}\\
\end{bmatrix}
$$

$$
\star \;\;\; 2X^TXh - 2X^TY = \begin{bmatrix}
    2m\sum{x_i^2} + 2b\sum{x_i} - 2\sum{x_iy_i} \\
    2m\sum{x_i} + 2bn - 2\sum{y_i}\\
\end{bmatrix} = \begin{bmatrix}
    \sum{-2x_i(y_i - mx_i - b)} \\
    \sum{-2(y_i-mx_i - b)}\\
\end{bmatrix} = \begin{bmatrix}
     \frac{\partial E}{\partial m} \\
      \frac{\partial E}{\partial b} \\
\end{bmatrix}
$$


## 3.1 Compute the gradient of loss function with respect to parameters 
## (Choose one between two 3.1 questions)
We define loss funtion $E$ as 
$$
E(m, b) = \sum_{i=1}^{n}{(y_i - mx_i - b)^2}
$$
and we define vertex $Y$, matrix $X$ and vertex $h$ :
$$
Y =  \begin{bmatrix}
    y_1 \\
    y_2 \\
    ... \\
    y_n
\end{bmatrix}
,
X =  \begin{bmatrix}
    x_1 & 1 \\
    x_2 & 1\\
    ... & ...\\
    x_n & 1 \\
\end{bmatrix},
h =  \begin{bmatrix}
    m \\
    b \\
\end{bmatrix}
$$

Proves that 
$$
E = Y^TY -2(Xh)^TY + (Xh)^TXh
$$

$$
\frac{\partial E}{\partial h} = 2X^TXh - 2X^TY
$$

TODO Please use latex （refering to the latex in problem may help）

TODO Proof：

## 3.2  Linear Regression
### Solve equation $X^TXh = X^TY $ to compute the best parameter for linear regression.

In [20]:
#TODO implement linear regression 
'''
points: list of (x,y) tuple
return m and b
'''
def linearRegression(points):
    X = [[point[0], 1] for point in points]
    Y = [[point[1]] for point in points]
    Xt = transpose(X)
    left = matxMultiply(Xt, X)
    right = matxMultiply(Xt, Y)
    return gj_Solve(left, right)

## 3.3 Test your linear regression implementation

In [21]:
#TODO Construct the linear function
# y = 2 * x + 4
#TODO Construct points with gaussian noise
import random
points = list()
sample = 100000
for x in range(sample):
    y = 2 * x + 4 +  random.gauss(0, 1)
    points.append((x, y))
#TODO Compute m and b and compare with ground truth
m, b = linearRegression(points)
print "m = 2, ", m[0]
print "b = 4, ", b[0]

m = 2,  1.99999985882
b = 4,  4.00685311185
